# Install pyrecdp from github

In [ ]:
!pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

# Install jdk for pyspark running

In [ ]:
!DEBIAN_FRONTEND=noninteractive apt-get install -y openjdk-8-jre

# Prepare test data

In [ ]:
%mkdir -p /content/test_data
%cd /content/test_data
!wget https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/llm_data/arxiv_sample_100.jsonl

# Import quality classifier function

In [5]:
from pyrecdp.primitives.llmutils import quality_classifier_spark, quality_classifier
from pyrecdp.core import SparkDataProcessor

# Specify variables

In [6]:
data_file = '/content/test_data/arxiv_sample_100.jsonl'
save_path = '/content/test_data/output/quality_classifier'

# Load data

In [7]:
rdp = SparkDataProcessor()
spark = rdp.spark
spark_df = spark.read.json(data_file)
spark_df.show()

Will assign 1 cores and 10386 M memory for spark
per core memory size is 10.143 GB and shuffle_disk maximum capacity is 8589934592.000 GB
+--------------------+--------------------+
|                meta|                text|
+--------------------+--------------------+
|{2203.15369, en, ...|\section{Introduc...|
|{math/9807097, en...|\section{Introduc...|
|{2008.06948, en, ...|\section{Introduc...|
|{cond-mat/9807071...|\section{Introduc...|
|{2210.10650, en, ...|\section{\label{s...|
|{astro-ph/9807119...|\section{Introduc...|
|{2111.03152, en, ...|\section{Introduc...|
|{1606.04992, en, ...|\n\n\section{Intr...|
|{1608.03404, en, ...|\section{introduc...|
|{1904.10101, en, ...|\section{Introduc...|
|{cond-mat/9807275...|\section{Introduc...|
|{2109.05334, en, ...|\section{Introduc...|
|{1512.06966, en, ...|\section{Introduc...|
|{2112.04926, en, ...|\section{Introduc...|
|{2202.01000, en, ...|\section{Introduc...|
|{2209.13421, en, ...|\section{Introduc...|
|{1103.5603, en, 2...|\sec

# Process the 'text' column and generate doc score which will determine whether the row kept or not(spark dataframe interface)

In [8]:
quality_classifier_df = quality_classifier_spark(spark_df)
quality_classifier_df.show()

2023-10-08 01:27:03.985 | INFO     | pyrecdp.primitives.operations.text_qualityscorer:prepare_model:122 - Preparing scorer model in [/root/.cache/recdp/models/gpt3_quality_model]...


model_name is gpt3


2023-10-08 01:27:18.945 | INFO     | pyrecdp.primitives.operations.text_qualityscorer:predict:252 - Start scoring dataset...


+--------------------+--------------------+--------------------+-----------+
|                meta|                text|           doc_score|should_keep|
+--------------------+--------------------+--------------------+-----------+
|{2203.15369, en, ...|\section{Introduc...|  0.9999999999996102|          1|
|{math/9807097, en...|\section{Introduc...|                 1.0|          1|
|{2008.06948, en, ...|\section{Introduc...|  0.9015197498942397|          0|
|{cond-mat/9807071...|\section{Introduc...|    0.99971976667992|          1|
|{2210.10650, en, ...|\section{\label{s...|                 1.0|          1|
|{astro-ph/9807119...|\section{Introduc...|0.001319136449002...|          0|
|{2111.03152, en, ...|\section{Introduc...|                 0.0|          0|
|{1606.04992, en, ...|\n\n\section{Intr...|                 1.0|          1|
|{1608.03404, en, ...|\section{introduc...|                 1.0|          1|
|{1904.10101, en, ...|\section{Introduc...|  0.9999979720467516|          1|

# Process the 'text' column and generate doc score and save data as parquet format.

In [13]:
quality_classifier(data_file, save_path, data_file_type="jsonl")
!ls $save_path

init spark
Will assign 1 cores and 10386 M memory for spark


2023-10-08 01:29:36.178 | INFO     | pyrecdp.primitives.operations.text_qualityscorer:prepare_model:122 - Preparing scorer model in [/root/.cache/recdp/models/gpt3_quality_model]...


per core memory size is 10.143 GB and shuffle_disk maximum capacity is 8589934592.000 GB
execute with spark started ...
model_name is gpt3
real_model_path is /root/.cache/recdp/models/gpt3_quality_model


2023-10-08 01:29:39.334 | INFO     | pyrecdp.primitives.operations.text_qualityscorer:predict:252 - Start scoring dataset...


execute with spark took 11.547022748000018 sec
part-00000-088e9ef7-8cf8-4987-a50b-a1775936ce5c-c000.snappy.parquet  _SUCCESS
